In [ ]:
%store -r WEAVIATE_IP
from boto3 import Session

session = Session()
credentials = session.get_credentials()
current_credentials = credentials.get_frozen_credentials()

AWS_ACCESS_KEY = current_credentials.access_key
AWS_SECRET_KEY = current_credentials.secret_key
AWS_SECRET_TOKEN = current_credentials.token
print(f"AWS_ACCESS_KEY:\t{AWS_ACCESS_KEY}")
print(f"AWS_SECRET_KEY:\t{AWS_SECRET_KEY}")
print(f"AWS_SECRET_TOKEN:\t{AWS_SECRET_TOKEN}")
print(f"WEAVIATE_IP:\t{WEAVIATE_IP}")

## Connect

In [ ]:
import weaviate

client = weaviate.connect_to_custom(
    http_host=WEAVIATE_IP, http_port="8080",  http_secure=False,
    grpc_host=WEAVIATE_IP, grpc_port="50051", grpc_secure=False,

    headers={
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
        "X-AWS-Session-Token": AWS_SECRET_TOKEN,
    }
)

client.is_ready()

## Data Preview

In [ ]:
# Get the collection
movies = client.collections.get("Movies")

response = movies.query.fetch_objects(limit=5)

for o in response.objects:
    # Print the title and release year (note the release date is a datetime object)
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        f"https://www.themoviedb.org/movie/{o.properties['tmdb_id']}",
        o.properties["poster_url"]
    )

## Title search

In [ ]:
import weaviate.classes.query as wq

movies = client.collections.get("Movies")

# Perform query
response = movies.query.near_text(
    query="A joyful holiday film",
    target_vector="title",  # The target vector to search against
    limit=5,
    # return_properties=["title", "release_date", "tmdb_id", "poster_url"],
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    # Print the title and release year (note the release date is a datetime object)
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        o.properties["tmdb_id"],
        o.properties["poster_url"]
    )

    # Print the distance of the object from the query
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )

In [ ]:
import weaviate.classes.query as wq

# Get the collection
movies = client.collections.get("Movies")

# Perform query
response = movies.query.near_text(
    query="A joyful holiday film",
    target_vector="overview",  # The target vector to search against
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    # Print the title and release year (note the release date is a datetime object)
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        o.properties["tmdb_id"],
        o.properties["poster_url"]
    )

    # Print the distance of the object from the query
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )

In [ ]:
client.close()